In [ ]:
import os
import json
import requests
import base64
import time
from pathlib import Path
from typing import List, Dict, Any

def find_sqlite_files(base_dir: str) -> List[Dict[str, str]]:
    """
    Tìm tất cả các file SQLite trong thư mục con của thư mục gốc.
    
    Args:
        base_dir: Đường dẫn đến thư mục gốc
        
    Returns:
        Danh sách các từ điển chứa {db_id, file_path}
    """
    sqlite_files = []
    
    # Duyệt qua tất cả các thư mục con
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        
        # Chỉ xử lý các thư mục
        if os.path.isdir(folder_path):
            # Tìm file .sqlite trong thư mục
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.sqlite'):
                    file_path = os.path.join(folder_path, file_name)
                    sqlite_files.append({
                        'db_id': folder_name,
                        'file_path': file_path
                    })
                    break  # Chỉ lấy file sqlite đầu tiên trong mỗi thư mục
    
    return sqlite_files

def encode_sqlite_file(file_path: str) -> str:
    """
    Đọc file SQLite và mã hóa Base64.
    
    Args:
        file_path: Đường dẫn đến file SQLite
        
    Returns:
        Chuỗi Base64 của file
    """
    with open(file_path, "rb") as file:
        sqlite_binary = file.read()
        return base64.b64encode(sqlite_binary).decode('utf-8')

def process_database(file_info: Dict[str, str], api_url: str, output_dir: str, timeout: int = 3000) -> bool:
    """
    Xử lý một cơ sở dữ liệu và lưu kết quả.
    
    Args:
        file_info: Thông tin về file SQLite
        api_url: URL của API schema-enrichment
        output_dir: Thư mục đầu ra
        timeout: Thời gian chờ tối đa (giây)
        
    Returns:
        True nếu thành công, False nếu thất bại
    """
    db_id = file_info['db_id']
    file_path = file_info['file_path']
    
    print(f"Đang xử lý cơ sở dữ liệu: {db_id}")
    
    try:
        # Mã hóa file SQLite
        sqlite_base64 = encode_sqlite_file(file_path)
        
        # Tạo payload
        payload = {
            "connection_payload": {
                "file": sqlite_base64,
                "dbType": "sqlite"
            }
        }
        
        # Gửi yêu cầu đến API
        response = requests.post(api_url, json=payload, timeout=timeout)
        
        # Kiểm tra xem yêu cầu có thành công không
        if response.status_code == 200:
            result = response.json()
            
            # Tạo tên file đầu ra
            output_file = os.path.join(output_dir, f"{db_id}.json")
            
            # Lưu kết quả vào file JSON
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
                
            print(f"✅ Đã lưu kết quả cho {db_id} vào {output_file}")
            return True
        else:
            print(f"❌ Lỗi khi xử lý {db_id}: {response.status_code} - {response.text}")
            return False
            
    except Exception as e:
        print(f"❌ Lỗi ngoại lệ khi xử lý {db_id}: {str(e)}")
        return False

def main():
    # Cấu hình
    spider_database_path = "./database"
    api_url = "http://127.0.0.1:8383/schema-enrichment"
    output_dir = "logs_schema_enrich"
    
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)
    
    # Tìm tất cả file SQLite
    sqlite_files = find_sqlite_files(spider_database_path)
    print(f"Tìm thấy {len(sqlite_files)} cơ sở dữ liệu SQLite")
    
    # Thống kê
    total = len(sqlite_files)
    success = 0
    failed = 0
    
    # Xử lý từng cơ sở dữ liệu
    for i, file_info in enumerate(sqlite_files):
        print(f"\nXử lý cơ sở dữ liệu {i+1}/{total}: {file_info['db_id']}")
        
        # Xử lý cơ sở dữ liệu
        result = process_database(file_info, api_url, output_dir)
        
        if result:
            success += 1
        else:
            failed += 1
        
        # Tạm nghỉ giữa các yêu cầu để tránh quá tải
        if i < total - 1:
            time.sleep(2)
    
    # In thống kê
    print(f"\n===== Kết quả =====")
    print(f"Tổng số: {total}")
    print(f"Thành công: {success}")
    print(f"Thất bại: {failed}")

if __name__ == "__main__":
    main()

In [9]:
# json dump result
import json
with open('schema_enrichment_results.json', 'w') as f:
    json.dump(response.json(), f, indent=2)

In [ ]:
import json
import base64
from typing import List, Dict, Any

# Import từ service.py của bạn
from core.services import get_schema, execute_sql, validate_connection_payload
from core.templates import SCHEMA_ENRICHMENT_TMPL
from core.utils import show_prompt

def get_table_examples(connection_payload, table_name, columns, limit=3, max_value_length=50):
    """Lấy các giá trị ví dụ từ bảng được chỉ định theo thứ tự cột cụ thể"""
    try:
        # Lấy tên cột từ danh sách columns (bỏ kiểu dữ liệu và các thông tin khác)
        column_names = []
        for col_def in columns:
            # Tách tên cột từ chuỗi định nghĩa (ví dụ: "id INTEGER PRIMARY KEY" -> "id")
            col_name = col_def.split()[0]
            column_names.append(col_name)
        
        # Tạo câu truy vấn SELECT với các cột cụ thể
        columns_query = ", ".join(column_names)
        query = f"SELECT {columns_query} FROM {table_name} LIMIT {limit}"
        
        result = execute_sql(connection_payload, query)
        
        # Kiểm tra lỗi
        if result.get("error"):
            return []
        
        # Truy cập trực tiếp đến data
        data = result.get("data", [])
        
        # Định dạng dữ liệu theo kiểu CSV với thứ tự cột giống như trong cấu trúc bảng
        csv_rows = []
        for record in data:
            # Tạo một hàng với các giá trị theo thứ tự cột
            row_values = []
            for col_name in column_names:
                value = record.get(col_name, "")
                # Chuyển đổi giá trị thành chuỗi
                str_value = str(value) if value is not None else "NULL"
                
                # Cắt ngắn chuỗi nếu quá dài
                if len(str_value) > max_value_length:
                    str_value = str_value[:max_value_length] + "..."
                
                # Thêm dấu ngoặc kép nếu giá trị chứa dấu phẩy
                if "," in str_value:
                    str_value = f'"{str_value}"'
                    
                row_values.append(str_value)
            
            # Nối các giá trị bằng dấu phẩy để tạo hàng CSV
            csv_rows.append(", ".join(row_values))
        
        return csv_rows
    except Exception as e:
        print(f"Error getting examples for table {table_name}: {str(e)}")
        return []

def create_table_relationship_arrays(table_details):
    """Tạo các mảng quan hệ của bảng với bảng chính ở vị trí đầu tiên"""
    # Khởi tạo dictionary để lưu mối quan hệ của từng bảng
    table_relationships = {}
    
    # Duyệt qua mỗi bảng trong schema
    for table in table_details:
        table_name = table["tableIdentifier"]
        # Khởi tạo mảng rỗng cho mỗi bảng
        table_relationships[table_name] = []
        
    # Duyệt lại từng bảng để tìm các mối quan hệ
    for table in table_details:
        table_name = table["tableIdentifier"]
        
        # Duyệt qua từng cột trong bảng
        for column in table["columns"]:
            if "relations" in column and column["relations"]:
                # Nếu cột có quan hệ, thêm bảng liên kết vào mảng
                for relation in column["relations"]:
                    related_table = relation["tableIdentifier"]
                    
                    # Thêm mối quan hệ hai chiều
                    if related_table not in table_relationships[table_name]:
                        table_relationships[table_name].append(related_table)
                    if table_name not in table_relationships[related_table]:
                        table_relationships[related_table].append(table_name)
    
    # Chuyển đổi từ dictionary thành mảng 2 chiều
    relationship_arrays = []
    for table_name, related_tables in table_relationships.items():
        # Tạo một mảng con với bảng chính ở vị trí đầu tiên
        array = [table_name] + related_tables
        relationship_arrays.append(array)
    
    # Đếm số lần xuất hiện của mỗi bảng trong tất cả các mảng quan hệ
    table_occurrence_count = {}
    for arr in relationship_arrays:
        for table in arr:
            if table in table_occurrence_count:
                table_occurrence_count[table] += 1
            else:
                table_occurrence_count[table] = 1
    
    # Tạo mảng chứa các bảng xuất hiện từ 2 lần trở lên
    duplicate_tables = [table for table, count in table_occurrence_count.items() if count >= 2]
    
    # Thêm mảng các bảng xuất hiện nhiều lần vào kết quả
    result = {
        "relationship_arrays": relationship_arrays,
        "duplicate_tables": duplicate_tables
    }
    
    return result

def create_schema_enrichment_prompts(table_details, table_groups, connection_payload):
    """Tạo các prompt cho việc phân tích ngữ nghĩa của schema với example values"""
    prompts = []
    
    for group in table_groups:
        main_table = group[0]
        related_tables = group[1:]
        
        # Get table structures and examples
        table_structures = {}
        table_examples = {}
        
        for table in table_details:
            if table["tableIdentifier"] in group:
                # Get structure
                columns = []
                for column in table["columns"]:
                    column_type = column["columnType"]
                    pk_indicator = " PRIMARY KEY" if column["isPrimaryKey"] else ""
                    columns.append(f"{column['columnIdentifier']} {column_type}{pk_indicator}")
                table_structures[table["tableIdentifier"]] = columns
                
                # Get examples with the same column order
                examples = get_table_examples(connection_payload, table["tableIdentifier"], columns)
                table_examples[table["tableIdentifier"]] = examples
        
        # Format the table group information
        group_info = f"Table group: {group}\n"
        for table_name in group:
            if table_name in table_structures:
                columns_str = ", ".join(table_structures[table_name])
                group_info += f"- {table_name}: ({columns_str})\n"
                
            # Add example values if available
            if table_examples.get(table_name):
                group_info += f"\n* Example Values for {table_name} (CSV format):\n"
                # First add column headers
                column_headers = ", ".join([col.split()[0] for col in table_structures[table_name]])
                group_info += f"{column_headers}\n"
                
                # Then add data rows
                for example in table_examples[table_name]:
                    group_info += f"{example}\n"
                group_info += "\n"
        
        # Create the prompt with simplified structured response format
        prompt = SCHEMA_ENRICHMENT_TMPL.format_messages(
            group_info=group_info
        )
    
        prompts.append({
            "main_table": main_table,
            "related_tables": related_tables,
            "prompt": prompt
        })
    
    return prompts

def parse_llm_json_response(response_text):
    try:
        # Tìm JSON trong phản hồi
        json_start = response_text.find('```json')
        if json_start != -1:
            # Nếu JSON được bọc trong dấu ```json ... ```
            json_content = response_text[json_start + 7:]  # +7 để bỏ qua ```json
            json_end = json_content.find('```')
            if json_end != -1:
                json_content = json_content[:json_end].strip()
            return json.loads(json_content)
        else:
            # Thử tìm '{'
            json_start = response_text.find('{')
            if json_start != -1:
                json_end = response_text.rfind('}') + 1
                if json_end > json_start:
                    json_content = response_text[json_start:json_end].strip()
                    return json.loads(json_content)
        
        # Thử phân tích toàn bộ chuỗi phản hồi
        return json.loads(response_text)
    
    except json.JSONDecodeError as e:
        print(f"Lỗi phân tích JSON: {e}")
        return None
    except Exception as e:
        print(f"Lỗi không mong đợi: {e}")
        return None

# Kết nối SQLite database
sqlite_file_path = r"PUT_YOUR_PATH to database ./database"

connection_payload = {
    "dbType": "sqlite",
    "file": None  # Sẽ được điền sau khi đọc file
}

# Đọc SQLite file và encode base64
try:
    with open(sqlite_file_path, "rb") as file:
        file_content = file.read()
        connection_payload["file"] = base64.b64encode(file_content).decode('utf-8')
except Exception as e:
    print(f"Error reading SQLite file: {str(e)}")
    print("Please provide a valid SQLite file path.")

# Kiểm tra connection payload
is_valid, error_msg = validate_connection_payload(connection_payload)
if not is_valid:
    print(f"Invalid connection payload: {error_msg}")
else:
    # Lấy schema của database
    table_details = get_schema(connection_payload)
    
    # Tạo các mảng quan hệ bảng
    table_groups = create_table_relationship_arrays(table_details)["relationship_arrays"]
    
    # Tạo các prompt phân tích ngữ nghĩa với example values
    prompts = create_schema_enrichment_prompts(table_details, table_groups, connection_payload)
    
    
    # In ví dụ prompt
    if prompts:
        print("Example prompt for the first table group:")
        print(prompts[0]["prompt"])
        print("\nTotal prompts generated:", len(prompts))
        print("All prompts saved to 'schema_enrichment_prompts.json'")

In [ ]:
print(create_table_relationship_arrays(table_details)["duplicate_tables"])

In [ ]:

for prompt in prompts:
    result = []
    show_prompt(prompt["prompt"])
    chat_response = llm.chat(prompt["prompt"])
    semantic = parse_llm_json_response(chat_response.message.content)
    print(semantic)



In [ ]:
semantic = parse_llm_json_response(chat_response.message.content)
print(semantic)